# Keras Scikit-learn wrapper class for utilizing cross-validation, grid search etc.
### Dr. Tirthajyoti Sarkar, Fremont, CA 94536

Keras offer a couple of special wrapper classes - for regression and classification - to utilize the full power of Scikit-learn cross-validation, model selection, pipelining, and grid search abilities.

To learn more, see this link: https://keras.io/scikit-learn-api/

In this notebook, we will show demo example of using simple k-fold cross-validation and grid search with a Keras classifier model.

**NOTE**: If you have latest version of NumPy, you will get lots of warning with Keras. You can supress them in the notebook if you want

In [1]:
import warnings
warnings.filterwarnings('ignore')

import os
os.environ['TF_CPP_MIN_LOG_LEVEL'] = '3'

In [2]:
# MLP for Pima Indians Dataset with 10-fold cross validation via sklearn
from keras.models import Sequential
from keras.layers import Dense
from keras.wrappers.scikit_learn import KerasClassifier
from sklearn.model_selection import StratifiedKFold
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import GridSearchCV
from sklearn.preprocessing import minmax_scale
import numpy as np

Using TensorFlow backend.


## Example of 10-fold cross-validation with a Keras classifier

### Function to create Keras model

In [3]:
def create_model():
    # create model
    model = Sequential()
    model.add(Dense(30, input_dim=8, activation='relu'))
    model.add(Dense(15, activation='relu'))
    model.add(Dense(1, activation='sigmoid'))
    # Compile model
    model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])
    return model

### Load pima indians dataset

In [4]:
dataset = np.loadtxt("..\Data\pima-indian-diabetes.csv", delimiter=",",skiprows=1)

### Split into input (X) and output (Y) variables, scale the X variables for better accuracy

In [5]:
X = dataset[:,0:8]
Y = dataset[:,8]

In [6]:
X_scaled = minmax_scale(X)

### Create the model using `KerasClassifier` class
Here, you pass on the `create_model` function, and other hyperparameters like epochs, and batch size.

Later, we will see how to also pass on those hyperparameters to this class for a grid search. 

In [12]:
model = KerasClassifier(build_fn=create_model, epochs=10, batch_size=32, verbose=0)

### Evaluate using 10-fold cross validation

In [13]:
num_folds = 10

In [14]:
kfold = StratifiedKFold(n_splits=num_folds, shuffle=True)

In [15]:
results = cross_val_score(model, X_scaled, Y, cv=kfold,verbose=2)

[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.


[CV]  ................................................................
[CV] ................................................. , total=   2.5s
[CV]  ................................................................


[Parallel(n_jobs=1)]: Done   1 out of   1 | elapsed:    2.4s remaining:    0.0s


[CV] ................................................. , total=   2.6s
[CV]  ................................................................
[CV] ................................................. , total=   2.8s
[CV]  ................................................................
[CV] ................................................. , total=   2.8s
[CV]  ................................................................
[CV] ................................................. , total=   3.0s
[CV]  ................................................................
[CV] ................................................. , total=   2.9s
[CV]  ................................................................
[CV] ................................................. , total=   3.0s
[CV]  ................................................................
[CV] ................................................. , total=   3.2s
[CV]  ................................................................
[CV] .

[Parallel(n_jobs=1)]: Done  10 out of  10 | elapsed:   29.7s finished


### Extract the mean of the cross-validation splits' scores

In [16]:
print(results.mean())

0.6901059493901218


## Example of an exhaustive grid search with a Keras classifier

### A model definition with extra arguments

In [17]:
def create_model_grid(optimizer='rmsprop', init='glorot_uniform'):
    # create model
    model = Sequential()
    model.add(Dense(12, input_dim=8, kernel_initializer=init, activation='relu'))
    model.add(Dense(8, kernel_initializer=init, activation='relu'))
    model.add(Dense(1, kernel_initializer=init, activation='sigmoid'))
    # Compile model
    model.compile(loss='binary_crossentropy', optimizer=optimizer, metrics=['accuracy'])
    return model

In [18]:
model_grid = KerasClassifier(build_fn=create_model_grid, verbose=0)

### Grid search epochs, batch size and optimizer

So, the exhaustive hyperparameter search space size is 3 x 3 x 2 x 2 = 36. 

Note that the **actual number of Keras runs will also depend on the number of cross-validation** we choose, as cross-validation will be used for each of these combinations.

In [19]:
optimizers = ['rmsprop', 'adam','sgd']
init = ['glorot_uniform', 'normal', 'uniform']
epochs = [5, 25]
batches = [8,64]

### Create a dictionary of parameters and pass that to the `GridSearchCV` function as `param_grid`
It is advisable to turn on verbosity at this stage to keep track of what's going on. Remember to keep the `verbose=0` for the main `KerasClassifier` class though, as you probably don't want to display all the gory details of training individual epcohs.

By default, GridSearchCV runs a 3-fold corss-validation if the `cv` parameter is not specified explicitly. The number of cross-validation is going to change to 5 in the next v0.22 of scikit-learn.

To know more about the Grid search function, see the following link: https://scikit-learn.org/stable/modules/generated/sklearn.model_selection.GridSearchCV.html

In [22]:
param_grid = dict(optimizer=optimizers, epochs=epochs, batch_size=batches, init=init)
grid = GridSearchCV(estimator=model_grid, param_grid=param_grid,verbose=2)

### Fit (search) the grid
This may take a while depending on the base model architecture, dataset size, and your hardware config.

In [23]:
grid_result = grid.fit(X_scaled, Y)

Fitting 3 folds for each of 36 candidates, totalling 108 fits
[CV] batch_size=8, epochs=5, init=glorot_uniform, optimizer=rmsprop ..


[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.


[CV]  batch_size=8, epochs=5, init=glorot_uniform, optimizer=rmsprop, total=   3.4s
[CV] batch_size=8, epochs=5, init=glorot_uniform, optimizer=rmsprop ..


[Parallel(n_jobs=1)]: Done   1 out of   1 | elapsed:    3.3s remaining:    0.0s


[CV]  batch_size=8, epochs=5, init=glorot_uniform, optimizer=rmsprop, total=   3.6s
[CV] batch_size=8, epochs=5, init=glorot_uniform, optimizer=rmsprop ..
[CV]  batch_size=8, epochs=5, init=glorot_uniform, optimizer=rmsprop, total=   3.6s
[CV] batch_size=8, epochs=5, init=glorot_uniform, optimizer=adam .....
[CV]  batch_size=8, epochs=5, init=glorot_uniform, optimizer=adam, total=   3.9s
[CV] batch_size=8, epochs=5, init=glorot_uniform, optimizer=adam .....
[CV]  batch_size=8, epochs=5, init=glorot_uniform, optimizer=adam, total=   3.9s
[CV] batch_size=8, epochs=5, init=glorot_uniform, optimizer=adam .....
[CV]  batch_size=8, epochs=5, init=glorot_uniform, optimizer=adam, total=   4.0s
[CV] batch_size=8, epochs=5, init=glorot_uniform, optimizer=sgd ......
[CV]  batch_size=8, epochs=5, init=glorot_uniform, optimizer=sgd, total=   3.9s
[CV] batch_size=8, epochs=5, init=glorot_uniform, optimizer=sgd ......
[CV]  batch_size=8, epochs=5, init=glorot_uniform, optimizer=sgd, total=   4.0s
[CV

[Parallel(n_jobs=1)]: Done 108 out of 108 | elapsed: 15.0min finished


### Print the best accuracy result

In [24]:
print("Best: %f using %s" % (grid_result.best_score_, grid_result.best_params_))

Best: 0.752604 using {'batch_size': 8, 'epochs': 25, 'init': 'glorot_uniform', 'optimizer': 'adam'}


### Extract all the results (both mean and std. dev for the cross-validation runs)

In [26]:
means = grid_result.cv_results_['mean_test_score']
stds = grid_result.cv_results_['std_test_score']
params = grid_result.cv_results_['params']

### Create a dataframe/table of all the runs with accuracy metric for analyzing later

In [27]:
import pandas as pd

d=pd.DataFrame(params)
d['Mean']=means
d['Std. Dev']=stds

In [28]:
d

,batch_size,epochs,init,optimizer,Mean,Std. Dev
0,8,5,glorot_uniform,rmsprop,0.656250,0.027805
1,8,5,glorot_uniform,adam,0.651042,0.024774
2,8,5,glorot_uniform,sgd,0.657552,0.028587
3,8,5,normal,rmsprop,0.651042,0.024774
4,8,5,normal,adam,0.651042,0.024774
5,8,5,normal,sgd,0.651042,0.024774
6,8,5,uniform,rmsprop,0.651042,0.024774
7,8,5,uniform,adam,0.651042,0.024774
8,8,5,uniform,sgd,0.651042,0.024774
9,8,25,glorot_uniform,rmsprop,0.684896,0.043303
